# PRELIMINARIES 

### STEP 1: run each of these in a separate terminal
#### TERMINAL 1:
export SERVER='carta-cav'

ssh -L 8081:localhost:8081 "$SERVER"

#### TERMINAL 2:
export SERVER='carta-cav'

ssh -L 26025:localhost:26025 "$SERVER"

### STEP 2: update the following two lines at the top of
#### multimodal_class.py

to be the appropriate paths on your system

path_to_VRPsolver = '/Users/dan/Documents/transit_webapp/'

path_to_multimodalsim = '/Users/dan/Documents/multimodal/'


### IMPORTS

In [ ]:
%load_ext autoreload
%autoreload 2
from multimodal_class import *

### INITIAL PARAMETERS

In [ ]:
params = {'csv_file_path': 'INPUTS/PRE_micro_2.csv',  # LOADS POPULATION DATA
          'modes':['drive','walk','gtfs','ondemand'],
          'time_window': [21600,36000], # in seconds. 6 AM to 10 AM
          'bnds':[np.array([-85.3394,  34.9458]), # bottom left
                  np.array([-85.2494,  35.0658])], # top right
          'groups_regions_geojson': 'INPUTS/regions2.geojson', # saved geojson file from geojson.io LOADS ONDEMAND REGIONS
          'background_congestion_file': 'INPUTS/background_traffic1.obj',  # OPTIONAL - LOADS BACKGROUND TRAFFIC
          ############ PRECOMPUTED GTFS DATA ########### 
          'gtfs_feed_file': 'INPUTS/carta_gtfs.zip',  # included in INPUTS folder
          'gtfs_precomputed_file': 'INPUTS/gtfs_trips.obj', # generated from gtfs  ONLY RUN ONCE
          'max_num_people':30,
         }

# SETUP SIMULATION

In [ ]:
# OPTION-1: FULL ONE-SHOT SETUP 
# BLAH = WORLD(params,full_setup = True)

# OPTION-2: STEP-BY-STEP SETUP
WORLD1 = WORLD(params)
WORLD1.initGRAPHSnFEEDS();
WORLD1.initNETWORKS();
WORLD1.initCONVERTER();
WORLD1.initSTATS(version='agrima_code'); ###### version = 'dan_code'
WORLD1.initONDEMAND(); 
WORLD1.initPEOPLE();
WORLD1.initBACKGROUND();


### FIT CONGESTION DEMAND MODELS

In [ ]:
from multimodal_class import *
WORLD1.fitModels(counts ={'num_counts':2,'num_per_count':1}); 


### PRELIMINARY PLOTS (OPTIONAL)

In [ ]:
from multimodal_class import *
WORLD1.plotPRELIMINARIES(include_demand_curves=True);

# RUN SIMULATION

In [ ]:
from multimodal_class import *
WORLD1.SIM(num_iters = 2, restart = True, compute_uncongested = True, force_takeall=False)

### GENERATE OUTPUTS

In [ ]:
from multimodal_class import *
WORLD1.generateOutputs(randomize=['money','switches']);

### PRINT OUTPUTS

In [ ]:
mode = 'gtfs'
dfs_to_show = [mode]
WORLD1.printOutputs(show_active=True,dfs_to_show = dfs_to_show,row_count = 10); 

### SAVE OUTPUTS

In [ ]:
WORLD1.saveOutputs('OUTPUTS/run7')

### GENERATE DASHBOARD

In [ ]:
from multimodal_class import *
WORLD1.generateLayers();
WORLD1.plotLAYERS(folder = 'DASH/dash1',overwrite = True,verbose=False)

### SHOW DASHBOARD

In [ ]:
from multimodal_class import *

WORLD1.DASH = DASHBOARD(WORLD1.dash_folder);
WORLD1.DASH.makeGrid();
WORLD1.DASH.addOutputs(WORLD1.OUTPUTS,use_active=True)
WORLD1.DASH.addImages(WORLD1.layers)
WORLD1.DASH.show()


In [ ]:
import plotly.io as pio
pio.write_html(WORLD1.DASH.fig, file='dash1.html', auto_open=True)

####  END OF CURRENT ####  END OF CURRENT ####  END OF CURRENT ####  END OF CURRENT ####  END OF CURRENT 

In [ ]:
# group = 'group0'
# RUNS = WORLD1.ONDEMAND.groups[group].runs
# for run in RUNS:
#     RUN = RUNS[run];
#     print(vars(RUN))
print(WORLD1.OUTPUTS['ondemand'])